# Mask, center, correct, calibrate and threshold 2D raw data

In [1]:
%matplotlib qt
import hyperspy.api as hs
import pyxem as pxm
import os, glob, gc
import numpy as np
import h5py
import matplotlib.pyplot as plt
from project_vars import *
from mg26825_4.mg26825_4_vars import *
import tqdm

In [9]:
DATA_ROOT = r"D:\Data\jf631\sed\mg26852-4"
data = []
# Local folders paths
data_folder = r"full_data_metadata"
mask_folder = r"mask"

# Name of files
data_wildcard = '**/*_data.hdf'
metadata_wildcard = '*_metadata.hdf'
apply_mask = True
mask_fname = r'mask_feb_2022_edited.npy'

threshold_ones = False
rotate_dp = False

crop_nav = True
crop_sig = True

affine_matrix = affine_matrix
rotation_angle = rotation_angle

nav_cal_dict = nav_cal_dict
recip_cal = recip_cal

print(nav_cal_dict)

{'80000': 9.58, '100000': 7.76, '150000': 5.13, '200000': 3.87, '250000': 3.09, '300000': 2.57, '600000': 1.29}


In [11]:
data = glob.glob(os.path.join(DATA_ROOT, data_folder, data_wildcard), recursive=True)
data.sort()

print(len(data))
data[:2], len(data)

316


(['D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\anneal\\20220127_132705\\20220127_132705_data_data.hdf',
  'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\anneal\\20220127_132955\\20220127_132955_data_data.hdf'],
 316)

In [12]:
# Filter only Au and Mo
data = [d for d in data if 'cal' in d]
data

['D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_142402\\20220126_142402_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_143235\\20220126_143235_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_143954\\20220126_143954_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_144455\\20220126_144455_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_144803\\20220126_144803_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_145038\\20220126_145038_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\au_xgrating_cal\\20220126_145259\\20220126_145259_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full_data_metadata\\moo3_cal\\20220128_093243\\20220128_093243_data_data.hdf',
 'D:\\Data\\jf631\\sed\\mg26852-4\\full

In [13]:
# Load a mask to cover the dead pixels and the joints between detectors
mask = np.load(os.path.join(DATA_ROOT, mask_folder, mask_fname))
plt.imshow(mask)

In [14]:
def get_magnification_from_file(metadata_fname):
    md = h5py.File(metadata_fname, 'r')
    mag = np.array(md['metadata/magnification'])
    return mag



for i, fname in tqdm.tqdm_notebook(enumerate(data[:])):
    dp_dir = os.path.dirname(fname)
    dp_name = os.path.basename(fname).split('.')[0]
    dp_name = dp_name.replace('_data_data', '')

    # Check if file already processed
    name_temp = f'{dp_name}_rebin_sig_2.hspy'
    path_to_check = os.path.join(dp_dir, name_temp)
    if os.path.exists(path_to_check):
        print(f"{dp_name} already processed!")
        continue

    # Load file
    f = h5py.File(fname, 'r')['Experiments/__unnamed__/data']
    dp = pxm.signals.electron_diffraction2d.ElectronDiffraction2D(np.array(f))


    # Apply mask
    if apply_mask:
        dp *= mask

    # Crop dp into divisible shape
    if crop_nav:
        dp = dp.inav[1:,1:]
    if crop_sig:
        dp = dp.isig[134:-61,144:-51]

    # Reset the offset
    dp.axes_manager.navigation_axes[0].offset = 0
    dp.axes_manager.navigation_axes[1].offset = 0
    dp.axes_manager.signal_axes[0].offset = 0
    dp.axes_manager.signal_axes[1].offset = 0

    # Get mean diffraction pattern to centre from there
    mean_dp = dp.mean()
    mean_dp = pxm.signals.electron_diffraction2d.ElectronDiffraction2D(mean_dp)
    centre = mean_dp.get_direct_beam_position(method = 'cross_correlate', radius_start = 1, radius_finish = 10)
    shifts = [[centre.data[0], centre.data[1]]]

    # Create shifts array and align and centre
    n_shifts = shifts * dp.axes_manager.navigation_shape[0] * dp.axes_manager.navigation_shape[1]
    n_shifts=np.array(n_shifts)
    new_shape = (dp.axes_manager.navigation_shape[1], dp.axes_manager.navigation_shape[0], 2)
    n_shifts = n_shifts.reshape(new_shape)
    dp.align2D(shifts = -n_shifts, crop = False)
    dp.center_direct_beam(method='interpolate', sigma=5, upsample_factor=4, kind='linear', half_square_width=10)

    # Calibrate
    # md_fname = glob.glob(os.path.join(dp_dir, metadata_wildcard))[0]
    # mag = get_magnification_from_file(md_fname)
    #nav_cal = nav_cal_dict[str(mag)]
    #dp.set_scan_calibration(nav_cal)
    dp.set_diffraction_calibration(recip_cal)

    # Save cropped
    name_temp = f'{dp_name}_cropped.hspy'
    dp.save(os.path.join(dp_dir, name_temp), overwrite=True)

    # Apply affine transforms (affine + rot)
    transform_mat = affine_matrix
    dp.apply_affine_transformation(transform_mat, keep_dtype=True)

    # Rotate
    if rotate_dp:
        continue
        # TODO: Add scipy.rotate with the dp.map function

    dp.set_diffraction_calibration(recip_cal)
    # Save
    name_temp = f'{dp_name}_processed.hspy'
    dp.save(os.path.join(dp_dir, name_temp), overwrite=True)

    # Save sum_dp
    sum_dp = dp.sum()
    name_temp = f'{dp_name}_sum_dp.hspy'
    sum_dp.save(os.path.join(dp_dir, name_temp), overwrite=True)
    sum_dp.change_dtype('float32')
    name_temp = f'{dp_name}_sum_dp.tiff'
    sum_dp.save(os.path.join(dp_dir, name_temp), overwrite=True)

    # Save rebinned data
    dp_nav_rebin = dp.rebin(scale=[2,2,1,1])
    name_temp = f'{dp_name}_rebin_nav_2.hspy'
    dp_nav_rebin.save(os.path.join(dp_dir, name_temp), overwrite=True)

    dp_sig_rebin = dp.rebin(scale=[1,1,2,2])
    name_temp = f'{dp_name}_rebin_sig_2.hspy'
    dp_sig_rebin.save(os.path.join(dp_dir, name_temp), overwrite=True)

    if threshold_ones:
        # Threshold the pixels with counts 1
        dp2 = dp.deecopy()
        dp2.data[dp2.data==1] = 0
        name_temp = f'{dp_name}_processed_threshold.hspy'
        dp2.save(os.path.join(dp_dir, name_temp), overwrite=True)
        del dp2

    # Clean up memory
    del dp
    gc.collect()
    print(f"File {i} {dp_name} finished.")

C:\Users\Sauron\AppData\Local\Temp\ipykernel_44724\1810937860.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, fname in tqdm.tqdm_notebook(enumerate(data[:])):


0it [00:00, ?it/s]

[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  1min  7.0s
File 0 20220126_142402 finished.
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  1min  6.4s
File 1 20220126_143235 finished.
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  1min  6.0s
File 2 20220126_143954 finished.
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  1min  6.5s
File 3 20220126_144455 finished.
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  1min  6.5s
File 4 20220126_144803 finished.
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Complet

  0%|          | 0/64516 [00:00<?, ?it/s]

  0%|          | 0/64516 [00:00<?, ?it/s]

...

  0%|          | 0/64516 [00:00<?, ?it/s]

  0%|          | 0/64516 [00:00<?, ?it/s]

  0%|          | 0/64516 [00:00<?, ?it/s]

  0%|          | 0/64516 [00:00<?, ?it/s]

  0%|          | 0/64516 [00:00<?, ?it/s]

In [31]:
name_temp = f'{dp_name}_processed.hspy'
dp = hs.load(os.path.join(dp_dir, name_temp),)
print(dp)


<ElectronDiffraction2D, title: , dimensions: (254, 254|320, 320)>
